## Installing the necessary pipelines

In [67]:
!python -m pip install --user --upgrade pip

!pip3 install pandas==0.23.4 matplotlib==3.0.3 scipy==1.2.1 scikit-learn==0.22 tensorflow==2.0 keras==1.2.2 --user

     |████████████████████████████████| 1.5 MB 7.0 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 20.2.4
    Uninstalling pip-20.2.4:
      Successfully uninstalled pip-20.2.4
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


## Installling kubeflow pipeline SDK

In [68]:
#install kubeflow pipeline sdk
!pip3 install kfp --upgrade --user

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 159 kB 7.1 MB/s eta 0:00:01
     |████████████████████████████████| 54 kB 3.8 MB/s  eta 0:00:01
  Created wheel for kfp: filename=kfp-1.1.2-py3-none-any.whl size=219914 sha256=515773ce7b039227b65007fd45bb420565d78d50eaad460304bcff5aa6577581
  Stored in directory: /home/jovyan/.cache/pip/wheels/4e/71/3f/79a5ffcfb1346020bd33519235f0f39a8ff6e3fc052b77552d
  Created wheel for kfp-server-api: filename=kfp_server_api-1.1.2rc1-py3-none-any.whl size=108990 sha256=4e4255455bafccea7f55755ed0abfb70c7dbbb0fa46b70e5917c2c614efe34c4
  Stored in directory: /home/jovyan/.cache/pip/wheels/3f/59/a4/9864fcf13058ac28e76ba8aafc044df66b2bc7559e12d8155f
Successfully built kfp kfp-server-api
  Attempting uninstall: kfp-server-api
    Found existing installation: kfp-server-api 1.0.1
    U

In [1]:
#checking if installation was successful
!which dsl-compile

In [2]:
# import libraries for pipeline
import kfp
from kfp import compiler
import kfp.dsl as dsl
import kfp.components as comp
import os
import subprocess
import json

In [3]:
# create  directory for outputs.
output_dir = "/home/jovyan/"

In [4]:
pwd

'/home/jovyan/g04-brazillian-commerce/pipeline'

In [5]:
#create preprocess pipeline
def download_dataset(data_path):
    
    # Import Libraries
    import pickle
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas==0.23.4'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.22'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'keras==1.2.2'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'nltk==3.2.5']) 
    import numpy as np
    import pandas as pd
    import re
    import nltk
    from nltk.corpus import stopwords
    nltk.download('stopwords')
    nltk.download('punkt')
    import tensorflow as tf
    import keras
    from tensorflow.keras.preprocessing.text import Tokenizer
    from keras.preprocessing.sequence import pad_sequences
    from sklearn.model_selection import train_test_split
    
    #downloading the dataset
    url = 'https://raw.githubusercontent.com/HamoyeHQ/g04-brazillian-commerce/master/data/olist_order_reviews_dataset.csv'
    
    
    #reading data from url
    review_df = pd.read_csv(url)
    #Save the whole_data as a pickle file to be used by the preprocess component.
    #review_df.to_pickle(f'{data_path}/data.pkl')
    #np.savez_compressed(f'{data_path}/dataset.npz', 
                       #data = review_df)
    with open(f'{data_path}/dataset','wb') as f:
        pickle.dump((review_df),f)
        
    return (print('Dataset Downloaded'))

In [6]:
download_dataset(output_dir)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Using TensorFlow backend.


Dataset Downloaded


In [7]:
#create preprocess pipeline
def clean_data(data_path):
    
    # Import Libraries
    import pickle
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas==0.23.4'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.22'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'keras==1.2.2'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'nltk==3.2.5']) 
    import numpy as np
    import pandas as pd
    import re
    import nltk
    from nltk.corpus import stopwords
    nltk.download('stopwords')
    nltk.download('punkt')
    import tensorflow as tf
    import keras
    from tensorflow.keras.preprocessing.text import Tokenizer
    from keras.preprocessing.sequence import pad_sequences
    from sklearn.model_selection import train_test_split
    
    # Load and unpack the downloaded dataset
    #review_df = pd.read_pickle(f'{data_path}/data.pkl')
    with open(f'{data_path}/dataset','rb') as f:
        review_df = pickle.load(f)
    
    
    #reading data
    #review_df = data
    
    # Preprocessing the reviews dataset
    review_data_title = review_df['review_comment_title']
    review_data = review_df.drop(['review_comment_title'],axis=1)

    # Dropping NaN values
    review_data  = review_data.dropna()
    review_data_title = review_data_title.dropna()

    # Resetting the reviews index and visualizing the data
    review_data = review_data.reset_index(drop=True)
    review_data_title = review_data_title.reset_index(drop=True)
    
    #Save the whole_data as a pickle file to be used by the clean_data component.
    #review_data.to_pickle(f'{data_path}/clean_data.pkl')
    with open(f'{data_path}/clean_data','wb') as f:
        pickle.dump((review_data),f)
        
    return (print('Dataset Cleaned'), review_data.shape)

In [8]:
clean_data(output_dir)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Dataset Cleaned


(None, (41753, 6))

In [12]:
#create preprocess pipeline
def map_review_rating(data_path):
    
    # Import Libraries
    import pickle
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas==0.23.4'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.22'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'keras==1.2.2'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'nltk==3.2.5']) 
    import numpy as np
    import pandas as pd
    import re
    import nltk
    from nltk.corpus import stopwords
    nltk.download('stopwords')
    nltk.download('punkt')
    import tensorflow as tf
    import keras
    from tensorflow.keras.preprocessing.text import Tokenizer
    from keras.preprocessing.sequence import pad_sequences
    from sklearn.model_selection import train_test_split
     
    # Load and unpack the downloaded dataset
    with open(f'{data_path}/clean_data','rb') as f:
        review_data = pickle.load(f)
    
    # Mapping the ratings
    review_data['Sentiment_rating'] = np.where(review_data.review_score > 3,1,0)
    
    # Removing neutral reviews 
    review_data = review_data[review_data.review_score != 3]
    
    #Save the whole_data as a pickle file to be used by the clean_data component.
    with open(f'{data_path}/rating_mapped','wb') as f:
        pickle.dump((review_data),f)
        
    return (print('Rating mapped!'), review_data.shape)

In [13]:
map_review_rating(output_dir)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Rating mapped!


(None, (38088, 7))

In [15]:
#create preprocess pipeline
def text_preprocessing(data_path):
    
    # Import Libraries
    import pickle
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas==0.23.4'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.22'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'keras==1.2.2'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'nltk==3.2.5']) 
    import numpy as np
    import pandas as pd
    import re
    import nltk
    from nltk.corpus import stopwords
    nltk.download('stopwords')
    nltk.download('punkt')
    import tensorflow as tf
    import keras
    from tensorflow.keras.preprocessing.text import Tokenizer
    from keras.preprocessing.sequence import pad_sequences
    from sklearn.model_selection import train_test_split
    
    # Load and unpack the downloaded dataset
    with open(f'{data_path}/rating_mapped','rb') as f:
        review_data = pickle.load(f)
    
    #making the target categorical
    labels = np.array(review_data['Sentiment_rating'])
    y = []
    for i in range(len(labels)):
        if labels[i] == 0:
            y.append(0)
        if labels[i] == 1:
            y.append(1)
    y = np.array(y)
    labels = tf.keras.utils.to_categorical(y, 3, dtype="float32")
    del y
    
    # Getting rid of stopwords, tokenizing and making the text lowercase
    comments = []
    stop_words = set(stopwords.words('portuguese'))
    
    for words in review_data['review_comment_message']:
        only_letters = re.sub("[^a-zA-Z]", " ",words)
        tokens = nltk.word_tokenize(only_letters) #tokenize the sentences
        lower_case = [l.lower() for l in tokens] #convert all letters to lower case
        filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) #Remove stopwords from the comments
        comments.append(' '.join(filtered_result))
    
    # making the text an array
    data = np.array(comments)

    max_words = 5000
    max_len = 200

    # Encoding and padding the texts
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(data)
    sequences = tokenizer.texts_to_sequences(data)
    encoded_data = pad_sequences(sequences, maxlen=max_len)
    
    
    #Save the whole_data as a pickle file to be used by the preprocess component.
    with open(f'{data_path}/model_data','wb') as f:
        pickle.dump((encoded_data, labels),f)
        
    
    return (print('Text preprocessing done!'), encoded_data.shape, labels.shape, review_data.shape)  

In [16]:
text_preprocessing(output_dir)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Text preprocessing done!


(None, (38088, 200), (38088, 3), (38088, 7))

In [41]:
#create train pipeline
def train(data_path):
    
    # Import Libraries
    import pickle
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas==0.23.4'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.22'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'keras==1.2.2'])
    
    import numpy as np
    import pandas as pd
    import re
    import tensorflow as tf
    import keras
    from tensorflow.keras.models import Sequential
    from tensorflow.keras import layers
    from tensorflow.keras.optimizers import RMSprop,Adam
    from tensorflow.keras.preprocessing.text import Tokenizer
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    from tensorflow.keras import regularizers
    from tensorflow.keras import backend as K
    from tensorflow.keras.callbacks import ModelCheckpoint
    from tensorflow.keras.callbacks import EarlyStopping
    from sklearn.model_selection import train_test_split
    
    # Load and unpack the clean_data
    with open(f'{data_path}/model_data','rb') as f:
        encoded_data, labels = pickle.load(f)
        
    
    max_words = 5000
    max_len = 200   
    # Separate the independent data (X) from the dependent data(y).
    #encoded_data, labels = clean_data
    
    #Splitting the data
    X_train, X_test, y_train, y_test = train_test_split(encoded_data,labels, random_state=0)
    
    # Building and training a bidirectional LSTM model
    classifier = Sequential()
    classifier.add(layers.Embedding(max_words, 40, input_length=max_len))
    classifier.add(layers.Bidirectional(layers.LSTM(20,dropout=0.6)))
    classifier.add(layers.Dense(3,activation='softmax'))
    classifier.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
    checkpoint = tf.keras.callbacks.ModelCheckpoint('/home/jovyan/', monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
    classifier.fit(X_train,y_train, epochs=25,validation_data=(X_test, y_test),callbacks=[checkpoint])
    
    classifier.summary()
    
    #checking the model loss
    test_loss, test_acc = classifier.evaluate(X_test, y_test, verbose=2)
    print('Model accuracy: ',test_acc) 
    
    #Save the model to the designated 
    classifier.save(f'{data_path}/sentiment_model.h5')
    
    #Save the test_data as a pickle file to be used by the predict component.
    with open(f'{data_path}/test_data', 'wb') as f:
        pickle.dump((X_test,  y_test), f)
    
    return (print('Training Done!'))  

In [31]:
pwd

'/home/jovyan/g04-brazillian-commerce/pipeline'

In [32]:
classifier = train(output_dir)

Train on 28566 samples, validate on 9522 samples
Epoch 1/25
28544/28566 [============================>.] - ETA: 0s - loss: 0.2937 - accuracy: 0.8774
Epoch 00001: val_accuracy improved from -inf to 0.92449, saving model to /home/jovyan/best_model.hdf5
28566/28566 [==============================] - 105s 4ms/sample - loss: 0.2936 - accuracy: 0.8774 - val_loss: 0.2025 - val_accuracy: 0.9245
Epoch 2/25
28544/28566 [============================>.] - ETA: 0s - loss: 0.2061 - accuracy: 0.9204
Epoch 00002: val_accuracy improved from 0.92449 to 0.92733, saving model to /home/jovyan/best_model.hdf5
28566/28566 [==============================] - 99s 3ms/sample - loss: 0.2062 - accuracy: 0.9204 - val_loss: 0.1933 - val_accuracy: 0.9273
Epoch 3/25
28544/28566 [============================>.] - ETA: 0s - loss: 0.1941 - accuracy: 0.9261
Epoch 00003: val_accuracy improved from 0.92733 to 0.93037, saving model to /home/jovyan/best_model.hdf5
28566/28566 [==============================] - 98s 3ms/sample 

In [42]:
def predict(data_path):
    
    import pickle
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'keras==1.2.2'])
    import numpy as np
    import tensorflow 
    import keras
    
    
    # Load the saved Keras model
    classifier = tensorflow.keras.models.load_model(f'{data_path}/sentiment_model.h5')

    # Load and unpack the test_data
    with open(f'{data_path}/test_data','rb') as f:
        test_data = pickle.load(f)
        
    # Separate the X_test from y_test.
    X_test,  y_test = test_data

    # make predictions.
    y_pred = classifier.predict(X_test)

    # create a threshold
    y_pred=(y_pred>0.5)
    
    with open(f'{data_path}/result.txt', 'w') as result:
        result.write(" Prediction: {}, Actual: {} ".format(y_pred,y_test.astype("int64")))
    
    print('Prediction has be saved successfully!')


In [34]:
predict(output_dir)

Prediction has be saved successfully!


In [43]:
# Create preprocess, train and predict lightweight components.
download_op = comp.func_to_container_op(download_dataset , base_image = "tensorflow/tensorflow:latest-gpu-py3")
clean_op = comp.func_to_container_op(clean_data , base_image = "tensorflow/tensorflow:latest-gpu-py3")
map_rating_op = comp.func_to_container_op(map_review_rating , base_image = "tensorflow/tensorflow:latest-gpu-py3")
text_preprocess_op = comp.func_to_container_op(text_preprocessing , base_image = "tensorflow/tensorflow:latest-gpu-py3")
train_op = comp.func_to_container_op(train , base_image = "tensorflow/tensorflow:latest-gpu-py3")
predict_op = comp.func_to_container_op(predict , base_image = "tensorflow/tensorflow:latest-gpu-py3")

## Build kubeflow pipeline

In [44]:
#Create a client to enable communication with the Pipelines API server.
client = kfp.Client()

In [45]:
import kubernetes as k8s
from kfp import compiler, dsl
from kubernetes.client import V1VolumeMount

# Define the pipeline
@dsl.pipeline(
   name='Sentiment Analysis Pipeline',
   description='An ML pipeline that performs sentiment analysis model training and prediction on customer reviews .'
)

# Define parameters to be fed into pipeline
def sentiment_container_pipeline(
    data_path: str,
    model_file: str
):
    #mount_folder = "/tmp"
    # Define volume to share data between components.
    vop = dsl.VolumeOp(
    name="create_volume",
    resource_name="data-volume", 
    size="1Gi", 
    modes=dsl.VOLUME_MODE_RWO)
    
    #op = dsl.PipelineVolume(pvc = vop, volume = )
    
    #volume = dsl.PipelineVolume(volume=k8s.client.V1Volume(
            #name=f"test-storage",
            #empty_dir=k8s.client.V1EmptyDirVolumeSource()))
    
    #op2.container.add_volume_mount(volume_mount=V1VolumeMount(mount_path=mount_folder,
                                                              #name=volume.name))
    #op2.after(op)
    
    # Create sentiment analysis preprocess component.
    sentiment_download_container = download_op(data_path) \
                                    .add_pvolumes({data_path: vop.volume})
    
     # Create sentiment analysis preprocess component.
    sentiment_clean_container = clean_op(data_path) \
                                    .add_pvolumes({data_path: sentiment_download_container.pvolume})
    
     # Create sentiment analysis preprocess component.
    sentiment_rating_container = map_rating_op(data_path) \
                                    .add_pvolumes({data_path: sentiment_clean_container.pvolume})
    
     # Create sentiment analysis preprocess component.
    sentiment_textpreprocess_container = text_preprocess_op(data_path) \
                                    .add_pvolumes({data_path: sentiment_rating_container.pvolume})
    
    # Create sentiment analysis training component.
    sentiment_training_container = train_op(data_path) \
                                    .add_pvolumes({data_path: sentiment_textpreprocess_container.pvolume})
                                    #{data_path: sentiment_textpreprocess_container.pvolume})
     #sentiment_textpreprocess_container.container.add_volume_mount(volume_mount=V1VolumeMount(mount_path=mount_folder,
                                                              #name=volume.name))
    #sentiment_textpreprocess_container.after(train_op)
    #sentiment_training_container = train_op(data_path) \
                                    #.add_pvolumes({data_path: sentiment_textpreprocess_container.pvolume})

    # Create sentiment analysis prediction component.
    sentiment_predict_container = predict_op(data_path) \
                                    .add_pvolumes({data_path: sentiment_training_container.pvolume})
    
    # Print the result of the prediction
    sentiment_result_container = dsl.ContainerOp(
        name="print_prediction",
        image='library/bash:4.4.23',
        pvolumes={data_path: sentiment_predict_container.pvolume},
        arguments=['cat', f'{data_path}/result.txt']
    )

## Run the pipeline

In [46]:
DATA_PATH = '/mnt'
MODEL_PATH='olist_ecommerce_sentiment_analysis.pkl'

In [47]:
pipeline_func = sentiment_container_pipeline

In [48]:
experiment_name = 'sentiment_analysis_kubeflow'
run_name = pipeline_func.__name__ + ' run'

arguments = {"data_path":DATA_PATH,
            "model_file":MODEL_PATH}

# Compile pipeline to generate compressed YAML definition of the pipeline.
kfp.compiler.Compiler().compile(pipeline_func, '{}.zip'.format(experiment_name))

# Submit pipeline directly from pipeline function
run_result = client.create_run_from_pipeline_func(pipeline_func, 
                                                  experiment_name=experiment_name, 
                                                  run_name=run_name, 
                                                  arguments=arguments)